## DIA 132: Exportar el Historial de Reportes a CSV o Excel (Filtrado)

Hoy aprenderás a permitir que el usuario exporte su historial de reportes a un archivo .csv o .xlsx, aplicando filtros activos como búsqueda por palabra clave y categoría.

Esto es útil para:

📑 Respaldo de datos

📈 Análisis fuera del sistema

📨 Compartir reportes por correo o Excel

🧩 1. Ruta Flask para exportar a CSV
python
Copiar
Editar
import csv
from flask import Response

@app.route('/exportar-historial.csv')
@login_required
def exportar_historial_csv():
    query = request.args.get('q', '').strip()
    categoria_filtro = request.args.get('categoria', '').strip()

    reportes_query = Reporte.query.filter_by(user_id=current_user.id)

    if query:
        reportes_query = reportes_query.filter(
            db.or_(
                Reporte.comentario.ilike(f'%{query}%'),
                Reporte.categoria.ilike(f'%{query}%'),
                Reporte.url.ilike(f'%{query}%')
            )
        )

    if categoria_filtro:
        reportes_query = reportes_query.filter(Reporte.categoria.ilike(f'%{categoria_filtro}%'))

    reportes = reportes_query.order_by(Reporte.fecha_generado.desc()).all()

    def generar_csv():
        datos = csv.writer(Response())
        yield ','.join(["ID", "Fecha", "Tipo", "Categoría", "Comentario", "URL"]) + '\n'
        for r in reportes:
            yield f'{r.id},{r.fecha_generado},{r.tipo},{r.categoria or ""},"{r.comentario or ""}",{r.url}\n'

    return Response(generar_csv(), mimetype='text/csv',
                    headers={"Content-Disposition": "attachment;filename=historial_reportes.csv"})
🧩 2. (Opcional) Exportar a Excel con pandas
python
Copiar
Editar
import pandas as pd
from io import BytesIO
from flask import send_file

@app.route('/exportar-historial.xlsx')
@login_required
def exportar_historial_excel():
    query = request.args.get('q', '').strip()
    categoria_filtro = request.args.get('categoria', '').strip()

    reportes_query = Reporte.query.filter_by(user_id=current_user.id)

    if query:
        reportes_query = reportes_query.filter(
            db.or_(
                Reporte.comentario.ilike(f'%{query}%'),
                Reporte.categoria.ilike(f'%{query}%'),
                Reporte.url.ilike(f'%{query}%')
            )
        )

    if categoria_filtro:
        reportes_query = reportes_query.filter(Reporte.categoria.ilike(f'%{categoria_filtro}%'))

    reportes = reportes_query.order_by(Reporte.fecha_generado.desc()).all()

    df = pd.DataFrame([{
        'ID': r.id,
        'Fecha': r.fecha_generado,
        'Tipo': r.tipo,
        'Categoría': r.categoria,
        'Comentario': r.comentario,
        'URL': r.url
    } for r in reportes])

    output = BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Reportes')

    output.seek(0)
    return send_file(output, as_attachment=True, download_name='historial_reportes.xlsx', mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
🧩 3. Botones en la vista historial_reportes.html
Colócalos arriba de la tabla:

html
Copiar
Editar
<a href="{{ url_for('exportar_historial_csv', q=request.args.get('q', ''), categoria=request.args.get('categoria', '')) }}">
  <button>📄 Exportar CSV</button>
</a>

<a href="{{ url_for('exportar_historial_excel', q=request.args.get('q', ''), categoria=request.args.get('categoria', '')) }}">
  <button>📊 Exportar Excel</button>
</a>